Test if 3d_2d result is right.

In [13]:
import cv2
import numpy as np
import json
import pickle

points_2d = np.loadtxt("./dataset/2d_array/0.txt")
points_3d = np.loadtxt("./dataset/3d_array/0.txt")

#with open("./dataset/2d_array/0.pkl", "rb") as f:
#    points_2d = pickle.load(f)

#with open("./dataset/3d_array/0.pkl", "rb") as f:
#    points_3d = pickle.load(f)


with open('./dataset/intrinsics.json') as f:
    intrinsics = json.load(f)
f_x = intrinsics['fx']
f_y = intrinsics['fy']
c_x = intrinsics['ppx']
c_y = intrinsics['ppy']


intrinsics_matrix = np.array([[f_x, 0, c_x],
                              [0, f_y, c_y],
                              [0, 0, 1]])

# Define the initial guess for the camera extrinsic parameters
R = np.eye(3)
t = np.zeros((3, 1))

# Run the PnP algorithm
success, R_est, t_est = cv2.solvePnP(points_3d, points_2d, intrinsics_matrix, None, flags=cv2.SOLVEPNP_ITERATIVE, rvec=R, tvec=t)

# Check if the PnP algorithm was successful
if not success:
    print('PnP algorithm failed to converge.')
else:
    # Project the 3D points onto the image plane using the estimated camera parameters
    projected_points, _ = cv2.projectPoints(points_3d, R_est, t_est, intrinsics_matrix, None)

    # Calculate the reprojection error
    error = cv2.norm(points_2d, projected_points.squeeze(), cv2.NORM_L2) / np.sqrt(points_2d.shape[0])

    # Check if the reprojection error is below a threshold
    if error > 1:
        print('2D and 3D points do not correspond.')
    else:
        print('2D and 3D points correspond.')

2D and 3D points correspond.


In [12]:
import cv2
import numpy as np
import json
import pickle

with open("./dataset/2d_array/0.pkl", "rb") as f:
    points_2d = pickle.load(f)

print(points_2d.shape)

(5076, 2)


In [1]:
import os
import pickle
import numpy as np
import open3d as o3d

# Specify the directory containing the .pkl files
directory = './point_cloud/'

# Create an empty list to store the point clouds
point_clouds = []

# Iterate over all .pkl files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):
        # Load the point cloud from the file
        with open(os.path.join(directory, filename), 'rb') as f:
            point_cloud = pickle.load(f)
        
        # Append the point cloud to the list
        point_clouds.append(point_cloud)

# Concatenate all point clouds into a single point cloud
combined_point_cloud = np.concatenate(point_clouds, axis=0)

# Convert the combined point cloud to an Open3D point cloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_point_cloud)

# Display the point cloud
o3d.visualization.draw_geometries([pcd])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


KeyboardInterrupt: 

Show point cloud

In [9]:
import pickle
import open3d as o3d

# Load the pkl file containing the point cloud
with open('point_cloud_world1.pkl', 'rb') as f:
    point_cloud = pickle.load(f)

# Create an Open3D point cloud object from the numpy array
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

Show rgb, mask and bbox.

In [10]:
import cv2
import numpy as np

# Load the RGB image
img = cv2.imread('./rgb/color_0.png')

# Convert the image to the HSV color space
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define the lower and upper bounds of the orange color in HSV
lower_orange = np.array([8, 50, 50])
upper_orange = np.array([11, 255, 255])

# Create a mask using the color threshold
mask = cv2.inRange(hsv, lower_orange, upper_orange)
mask = np.where(mask == 255, 1, mask)

contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the largest contour
largest_contour = max(contours, key=cv2.contourArea)

# Draw a bounding box around the largest contour
x, y, w, h = cv2.boundingRect(largest_contour)
result = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Create a new mask that is the same size as the original image and fill it with zeros
new_mask = np.zeros_like(mask)

# Set the values inside the bounding box to be the same as the values in the original mask
new_mask[y:y+h, x:x+w] = mask[y:y+h, x:x+w]

mask = np.zeros((new_mask.shape[0],new_mask.shape[1],3), dtype=int)

mask[:,:,0] = new_mask
mask[:,:,1] = new_mask
mask[:,:,2] = new_mask

# Apply the new mask to the original image to extract the orange wood cube within the bounding box

# Display the result
#cv2.imshow('Original Image', img)
cv2.imshow('Mask', new_mask)
cv2.imshow('Result', result)
cv2.imwrite(f"./mask/mask_{0}.png", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


Show where is the non-zero pixel in mask

In [ ]:
import cv2
import numpy as np
img = cv2.imread("./mask/mask_0.png")
b, g, r = cv2.split(img)
# Find the coordinates of pixels that have a different value between the channels
diff_pixels = np.where((r != 0))

# Print the coordinates of the different pixels
for i in range(len(diff_pixels[0])):
    print("Coordinate ({}, {}) has different pixel values between channels".format(diff_pixels[0][i], diff_pixels[1][i]))

Show mask only.

In [22]:
from PIL import Image
import cv2
import numpy as np
img = Image.open("./mask/mask_59.png")
img_array = np.array(img)
img_array[img_array == 1] = 255
img = Image.fromarray(img_array)
img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

# Display the image using OpenCV
cv2.imshow('Image', img_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()

Generate .pcd point cloud

In [38]:
import numpy as np
import open3d as o3d
from PIL import Image
import json

def create_point_cloud_from_rgbd_image(color, depth, intrinsic, extrinsic, mask):
    color = color * mask[:, :, np.newaxis]
    depth = depth * mask
    print(color)
    print(depth.shape)

    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        o3d.geometry.Image(color),
        o3d.geometry.Image(depth),
        depth_scale=1000,
        depth_trunc=1.0,
        convert_rgb_to_intensity=False
    )
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image,
        intrinsic,
        extrinsic
    )
    return pcd

# Load the color and depth images
color = o3d.io.read_image("./dataset/rgb_0.png")
depth = o3d.io.read_image("./dataset/depth_0.png")
mask = o3d.io.read_image("./dataset/mask_0.png")
mask = np.asarray(mask)
#mask[:] = 1
mask = mask.astype(bool)

with open('./dataset/intrinsics.json') as f:
    intrinsics = json.load(f)
f_x = intrinsics['fx']
f_y = intrinsics['fy']
c_x = intrinsics['ppx']
c_y = intrinsics['ppy']


# Load the camera intrinsic parameters
intrinsic = o3d.camera.PinholeCameraIntrinsic(
    width=1280,
    height=720,
    fx=f_x,
    fy=f_y,
    cx=c_x,
    cy=c_y
)

pose = np.load('./dataset/pose_0.npy')
print(pose)
R = pose[0:3, 0:3]
T = pose[0:3, 3]
extrinsic = np.eye(4)
extrinsic[:3, :3] = R
extrinsic[:3, 3] = T

# Create the point cloud
pcd = create_point_cloud_from_rgbd_image(color, depth, intrinsic, extrinsic, mask)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])
o3d.io.write_point_cloud("cube_point_cloud.ply", pcd)

[[ 6.88513871e-03  1.90047300e-03  9.99974491e-01 -1.84220698e-03]
 [-5.62283386e-01  8.26941415e-01  2.29987795e-03 -6.95060990e-05]
 [-8.26915950e-01 -5.62284878e-01  6.76221074e-03  9.99643111e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
(480, 640)
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


True

In [ ]:
import numpy as np
import open3d as o3d

# Load the PLY file
ply_file = o3d.io.read_point_cloud('cube_point_cloud.ply')
points = np.asarray(ply_file.points)

# Load the camera intrinsic and extrinsic parameters
intrinsics_matrix = np.array([[f_x, 0, c_x],
                              [0, f_y, c_y],
                              [0, 0, 1]])

print(T.shape)
extrinsics_matrix = np.hstack((R, np.expand_dims(T, -1)))

# Initialize a dictionary to store the correspondences
correspondences = {}

# Iterate over all 3D points and establish correspondences
for point_3d in points:
    # Transform the 3D point to camera coordinates
    point_3d_hom = np.hstack((point_3d, 1))
    point_camera_hom = extrinsics_matrix @ point_3d_hom
    point_camera = point_camera_hom[:3]

    # Project the 3D point onto the camera image plane
    point_hom = intrinsics_matrix @ point_camera
    point_2d_hom = point_hom / point_hom[2]
    point_2d = point_2d_hom[:2]

    # Store the correspondence between the 3D point and the corresponding 2D point
    correspondences[tuple(point_3d)] = tuple(point_2d)

# Print the correspondences
for point_3d, point_2d in correspondences.items():
    print(f'3D point: {point_3d} | 2D point: {point_2d}')

In [37]:
import cv2
import numpy as np

points_2d = []
points_3d = []
# Define the 2D and 3D points
for point_3d, point_2d in correspondences.items():
    points_2d.append(point_2d)
    points_3d.append(point_3d)

points_2d = np.array(points_2d)
points_3d = np.array(points_3d)

with open('./dataset/intrinsics.json') as f:
    intrinsics = json.load(f)
f_x = intrinsics['fx']
f_y = intrinsics['fy']
c_x = intrinsics['ppx']
c_y = intrinsics['ppy']


intrinsics_matrix = np.array([[f_x, 0, c_x],
                              [0, f_y, c_y],
                              [0, 0, 1]])

# Define the initial guess for the camera extrinsic parameters
R = np.eye(3)
t = np.zeros((3, 1))

# Run the PnP algorithm
success, R_est, t_est = cv2.solvePnP(points_3d, points_2d, intrinsics_matrix, None, flags=cv2.SOLVEPNP_ITERATIVE, rvec=R, tvec=t)

# Check if the PnP algorithm was successful
if not success:
    print('PnP algorithm failed to converge.')
else:
    # Project the 3D points onto the image plane using the estimated camera parameters
    projected_points, _ = cv2.projectPoints(points_3d, R_est, t_est, intrinsics_matrix, None)

    # Calculate the reprojection error
    error = cv2.norm(points_2d, projected_points.squeeze(), cv2.NORM_L2) / np.sqrt(points_2d.shape[0])

    # Check if the reprojection error is below a threshold
    if error > 1:
        print('2D and 3D points do not correspond.')
    else:
        print('2D and 3D points correspond.')

2D and 3D points correspond.


Convert mask image to txt

In [44]:
mask = o3d.io.read_image("./mask/mask_0.png")
mask = np.asarray(mask)
mask = mask[:,:,0]
mask = np.asarray(mask)
np.savetxt("mask.txt", mask)


Get extrinsics

In [55]:
import pyk4a as k4a
import json

config = k4a.Config(
    color_resolution=k4a.ColorResolution.RES_720P,
    depth_mode=k4a.DepthMode.WFOV_2X2BINNED,
    synchronized_images_only=True,
    camera_fps=k4a.FPS.FPS_30,
)
kinect = k4a.PyK4A(config)
# Initialize the Azure Kinect device
kinect.start()

# Get the calibration information for the device
calibration = kinect.calibration
capture = kinect.get_capture(-1)

# Get the extrinsic matrix between the color and depth cameras
extrinsics = calibration.get_extrinsic_parameters(k4a.calibration.CalibrationType.DEPTH,k4a.calibration.CalibrationType.COLOR)
# Save the extrinsic matrix to a file

rotation = extrinsics[0]
translation = extrinsics[1]

# Convert the matrices to lists for JSON serialization
rotation_list = rotation.flatten().tolist()
translation_list = translation.tolist()

# Create a dictionary to hold the extrinsics information
extrinsics_dict = {
    "rotation": rotation_list,
    "translation": translation_list
}

# Save the extrinsics to a JSON file
with open("./extrinsic/extrinsics_d_c.json", "w") as f:
    json.dump(extrinsics_dict, f)


Return 2d mask and 3d point cloud

In [12]:
import numpy as np
import cv2

# Load RGB image and depth map
rgb_image = cv2.imread('./rgb/color_0.png')
depth_map = cv2.imread('./depth_map/depth_map_0.png', cv2.IMREAD_GRAYSCALE)

# Load intrinsic matrix
intrinsic_matrix = np.loadtxt('./intrinsic_depth/intrinsicc_0.txt')

# Load mask image
mask = cv2.imread('./mask/mask_0.png', cv2.IMREAD_GRAYSCALE)

# Initialize list to store 3D coordinates of points
points = []
masks = []

# Loop over each pixel in the RGB image
for i in range(rgb_image.shape[0]):
    for j in range(rgb_image.shape[1]):
        # Check if the pixel is masked or not
        if mask[i, j]:
            # Convert pixel coordinate and depth value to 3D point
            homogeneous_pixel = np.array([j, i, 1])
            homogeneous_point = np.append(np.dot(np.linalg.inv(intrinsic_matrix), homogeneous_pixel), 1)
            depth = depth_map[i, j] / 1000.0  # convert from millimeters to meters
            point = homogeneous_point * depth
            points.append(point[:3])
            masks.append([j,i])

# Convert list to numpy array
point_cloud = np.array(points)
masks = np.array(masks)
# Save point cloud to disk
np.savetxt('./point_cloud.txt', point_cloud)
np.savetxt('./mask.txt', masks)

In [8]:
import matplotlib.pyplot as plt
import numpy as np

# Load the image
image = plt.imread('./rgb/color_0.png')

# Create an empty mask array
mask = np.zeros_like(image)

# Load the mask coordinates from text file
with open('mask.txt') as f:
    for line in f:
        x, y = map(round, line.strip().split())
        mask[y, x] = 1

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2)

# Display the image on the left subplot
ax1.imshow(image)
ax1.set_title('Original Image')

# Display the mask on the right subplot
ax2.imshow(mask, cmap='gray')
ax2.set_title('Mask')

# Show the figure
plt.show()


TypeError: float() argument must be a string or a number, not 'list'